# Lab de MongoDB - Introduccion

Las fuentes y recursos de este laboratorio provienen de 
- [mongodb-labs](https://github.com/mattdavis0351/mongodb-labs) por **Ishana Raina** y **Matthew Davis**

## Outline del Lab 1

- De un JSON, crear la base de datos al MongoDB Server por medio de MongoDB Database Tools
- Visualizar los datos de la base de datos
- Anhadir datos y entender su funcionamiento

## Crear base de datos por medio de JSON

Para poder cargar los datos de un JSON hacia MongoDB creando una base de datos, se tiene que usar el ejecutable `mongoimport`, este ejecutable(que es una instalacion aparte de MongoDB Server) nos pide:
<pre>
- --db          Nombre de la base de datos
- --collection  Nombre de la coleccion(tabla)
- --file        Localizacion del archivo
- --jsonArray   Indica que el JSON a importar es un arreglo
</pre>

Y con esto podremos cargar la base de datos a MongoDB

In [ ]:
mongoimport --db sample_mflix --collection movies --file "C:\Users<completar el path con el movies.json en la carpeta data>" --jsonArray

Para eliminar una base de datos hay que realizar lo siguiente
```bat
:: Ingrear a mongosh para manipular la base de datos
mongosh

:: Mostrar las bases de datos existentes
show databases

:: Seleccionar la base de datos a borrar
use <nombreBD>

:: Borrar la base de datos
db.dropDatabase()
```

## Pymongo

Para poder usar MongoDB en python, hay que descargar por medio de pip `pymongo`, driver para poder aceder al API de MongoDB y manipular la base de datos
Para instalarlo hay que ejecutar `pip install pymongo`, y se podra acceder a MongoDB desde python

In [ ]:
pip install pymongo

# Setup

Ahora con el driver instalado, hay que importarlo y usarlo

Recordar tener `mongod` corriendo en una terminal para que funcione

In [1]:
from pymongo import MongoClient
client = MongoClient()

# Confirmar la conexion a MongoDB
client.server_info()

# Conseguir los datos cargados

Ahora para poder conseguir los datos de una coleccion(`db.movies`), podemos usar distintas funciones para ello, como las siguientes:
<pre>
- find_one()                Recupera el primer documento dentro de la coleccion
- distinct('')              Recupera una sola vez los valores distintos segun la llave especificada
- find({})                  Recupera multiples documentos solo si cumplen con el 
                            filtro especificado(si esta vacio entonces recupera todos los documentos)
- find({}, {'filter': 1})   Lo mismo que "find()" pero con la capacidad de mostrar 
                            los documentos filtrando las llaves que querramos de los documentos
- aggregate(pipeline)       Recupera y agrupa los documentos segun los valores de la llave 
                            especificada por el pipeline, y aplica una opercion sobre los grupos formados
</pre>


In [31]:
# Setup de las variables a usar
db = client.sample_mflix
moviesCollection = db.movies

In [ ]:
# find_one()

result = db.movies.find_one()

result

In [ ]:
# distinct('')

uniqueNames = moviesCollection.distinct('name')
print(uniqueNames)

In [ ]:
# find({})

db = client.sample_mflix
moviesCollection = db.movies
allMovies = list(moviesCollection.find({}))

allMovies[0]

In [ ]:
# find({}, {'filter': 1})

# Recuperando todos los documentos pero solo con la llave 'name', e ignorando todos los demas
# Util para poder crear pipelines efectivas con metodos como el 'aggregate'
allMovies = list(moviesCollection.find({}, {'name': 1}))
allMovies[0]

In [ ]:
# find({'filter': {operator}})

# Usando el filtro con el operador 'regex' para encontrar peliculas que inician con el nombre 'Taylor'
taylorMovies = list(moviesCollection.find({'name': {'$regex': 'Taylor.*'}}))
taylorMovies

In [ ]:
# aggregate(pipeline)

# Agrupar las peliculas segun su nombre, y contar los elementos de cada grupo(o sea contar cuantos nombres se repiten)
pipeline = [
    {
        '$group': {
            '_id': '$name', # Criterio de agrupamiento
            'count': {'$sum': 1} # Operacion de agrupamiento(mismo formato de los demas operadores del API)
        }
    }
]

result = moviesCollection.aggregate(pipeline)
for item in result:
    print(f"Name: {item['_id'].ljust(18)} Count: {item['count']}") # .ljust(40) mejora el formato de presentacion(left-justify text)